In [ ]:
!pip install openai

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
from openai import OpenAI
from google.colab import userdata
import os

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
new = pd.read_csv('/content/drive/MyDrive/CDT Documents/cleaned_banktrack_annotated.csv')

In [ ]:
len(new)

165

In [ ]:
# Set the API key from Colab's user data secrets
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

client = OpenAI()

client.batches.create(
  input_file_id="file-abc123",
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

Batch(id='batch_687e5c75af9c8190857a81ee2f8a96b8', completion_window='24h', created_at=1753111669, endpoint='/v1/chat/completions', input_file_id='file-abc123', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1753198069, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
from datasets import DatasetDict
def undersample_majority_class(dataset_dict, label_column):
    # Extract train split
    np.random.seed(42)
    train_data = dataset_dict["train"]
    labels = np.array(train_data[label_column])
    # Identify majority and minority classes
    class_counts = np.bincount(labels)
    minority_class = np.argmin(class_counts)
    minority_count = np.min(class_counts)
    # Get indices for both classes
    minority_indices = np.where(labels == minority_class)[0]
    majority_indices = np.where(labels != minority_class)[0] # Q: what does [0] entail in this situation
    # Randomly select subset of majority class to match minority count
    undersampled_majority_indices = np.random.choice(
        majority_indices,
        size=minority_count,  # Match minority count
        replace=False  # No duplicates
    )
    # Combine selected majority + all minority samples
    new_indices = np.concatenate([undersampled_majority_indices, minority_indices])
    # Create new balanced dataset
    balanced_train = train_data.select(new_indices)

    # Create the output DatasetDict, only including 'validation' if it exists
    output_dict = {"train": balanced_train}
    if "validation" in dataset_dict:
        output_dict["validation"] = dataset_dict["validation"]

    return DatasetDict(output_dict)
# Usage
# Convert the pandas DataFrame 'new' to a Hugging Face DatasetDict
from datasets import Dataset
dataset_dict = DatasetDict({"train": Dataset.from_pandas(new)})

dataset = undersample_majority_class(dataset_dict,
    label_column="contains_debt_instrument_information"
)
# Verify
print("Original counts:", np.bincount(dataset_dict["train"]["contains_debt_instrument_information"]))
print("Balanced counts:", np.bincount(dataset["train"]["contains_debt_instrument_information"]))

Original counts: [ 55 110]
Balanced counts: [55 55]


Method 1

In [ ]:
# Step 1: Count words in 'text' column
# .astype(str) converts data type of column to string
# .str.split() splits the string by whitespace
# .apply(len) counts number of words in original string
new['word_count'] = new['text'].astype(str).str.split().apply(len)

# Step 2: Filter rows with more than 2000 words
# .copy() copies original dataframe
long_rows = new[new['word_count'] > 2000].copy()

# Step 3: Define the summarization function
# text is string of text I want to summarize
# prompt tells AI model what to do and limits response to 300 words
def summarize_text(text, max_words=300):
    prompt = f"""Summarize the following text in no more than {max_words} words.
Keep only the most essential details and avoid repetition.

Text:
{text}
"""
# sending request to OpenAI API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    # cleaned summary string is returned
    return response.choices[0].message.content.strip()

# Step 4: Apply summarization to the full text
# each entry in text column in long_rows df is converted to a string
# .apply() runs the summarize_text function on each text entry
long_rows['summary'] = long_rows['text'].astype(str).apply(lambda x: summarize_text(x, max_words=300))

# Assign summaries back to 'new' by matching index
matching_idx = long_rows.index.intersection(new.index)
new.loc[matching_idx, 'summary'] = long_rows.loc[matching_idx, 'summary']
print(long_rows)

                                                  text  \
51       Litigation Related to the Merger    In con...   
57       On June 14, 2018, Concho Resources Inc. (t...   
118   Stone Point Capital Co-investment in Atrium a...   

                     company      cik  item  \
51       CONCO RESOURCES INC  1358071  8.01   
57       CONCO RESOURCES INC  1358071  8.01   
118  CASTLEWOOD HOLDINGS LTD  1363829  1.01   

     contains_debt_instrument_information  word_count  \
51                                  False        5538   
57                                   True        2065   
118                                  True        2600   

                                               summary gpt_pred_raw  gpt_pred  \
51   Seven lawsuits were filed in late 2020 related...     52. True      True   
57   Concho Resources Inc. announced a public offer...     58. True      True   
118  Kenmare Holdings Ltd, a subsidiary of Enstar G...    119. True      True   

     gpt_pred_bool  true

Batch Prompting

In [ ]:
import openai
def classify_batch(batch_texts, start_idx=1):
    few_shot_prompt = """ laugh out loud

Examples:
1. "    Alpha Holding Company, LLC Disposition  On January 19, 2017, Concho Resources Inc. (the  Company ), through its wholly owned subsidiary, COG Operating LLC  ( Seller ), entered into a definitive securities purchase agreement (the  Purchase Agreement ) with Plains Pipeline, L.P. ( Purchaser ) to sell 100% of the issued and  outstanding Class A Units of Alpha Holding Company, LLC ( AHC ) (the  Disposition ), for aggregate consideration of approximately $800 million, subject to customary purchase price  adjustments, after closing of the transactions contemplated by the Class B Purchase Agreement (defined below) and repayment of the outstanding borrowings under the credit facility of Alpha Crude Connector, LLC, a wholly owned subsidiary of AHC.  Contemporaneously with the execution of the Purchase Agreement, Purchaser entered into a definitive purchase agreement with Frontier Midstream Solutions, LLC ( Frontier ) to purchase 100% of the issued and outstanding  Class B Units of AHC (the  Class  B Purchase Agreement ). Closing under the Purchase Agreement and the Class B Purchase Agreement is anticipated to occur contemporaneously during the  first half of 2017, subject to customary closing conditions, as further described below. Seller retains the right under the Limited Liability Company Agreement of AHC, subject to certain conditions, to purchase or cause AHC to purchase all of the  Class B Units of AHC prior to closing under the Purchase Agreement and the Class B Purchase Agreement (an  Option Exercise ). In the event of an Option Exercise, Seller will sell all of the Class A Units and  the Class B Units of AHC to Purchaser for an aggregate purchase price of approximately $1.215 billion. The Company expects the net proceeds to Seller in the event of an Option Exercise and sale to Purchaser of all of the Class A Units and  the Class B Units of AHC to be substantially similar to what Seller would otherwise receive under the Purchase Agreement in the absence of an Option Exercise.  Each partys obligation to consummate the Disposition is conditioned upon, among other things, (i) confirmation of the  counterparties representations and warranties as of the closing, (ii) the counterparties performance, in all material respects, of all covenants, (iii) the receipt of all required approvals (including approval under the  Hart-Scott-Rodino Antitrust Improvements Act of 1976), (iv) the absence of legal matters prohibiting the Disposition, and (v) the satisfaction or waiver by Frontier or Purchaser, as applicable, of certain conditions precedent to the  consummation of the transactions contemplated by the Class B Purchase Agreement.  " → False
2. "    As previously disclosed, between December 14, 2022 and December 16, 2022, IronNet, Inc. (the " Company ") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the " Initial Notes ") to a total of eight lenders, and on December 30, 2022, the Company issued a secured promissory note in the principal amount of $2.0 million (the " First   C5 Note ") to an entity affiliated with C5 Capital Limited (" C5 "), a beneficial owner of more than 5% of the Company's outstanding common stock.    In connection with the issuance of the First C5 Note, the Company and the holders of the Initial Notes agreed to amend and restate the Initial Notes to be substantially in the form to be issued to C5, which form was attached as an exhibit to the Current Report on Form  8-K  filed on January 4, 2023 (the " Restated Initial Notes "). This amendment and restatement occurred on January 11, 2023. The Restated Initial Notes continue to bear interest at a rate of 13.8% per annum from the respective dates of the Initial Notes, and the Restated Initial Notes are payable at scheduled maturity on June 30, 2023.    Upon entering into the First C5 Note, the Company and C5 also agreed to amend and restate the First C5 Note. Subsequently, the parties agreed that the amended and restated First C5 Note and any future notes to be issued in connection with additional bridge financing by C5 would be convertible into shares of the Company's common stock. On January 11, 2023, the Company issued to C5 an amended and restated convertible promissory note in the principal amount of $2.0 million (the " Restated C5 Note "). On January 12, 2023, the Company issued to C5 a new secured convertible promissory note in the principal amount of $3.0 million (the " Second C5 Note "). The Restated C5 Note and the Second C5 Note (collectively, the " C5 Notes "), in the aggregate principal amount of $5.0 million, are in substantially the form of secured convertible promissory note attached to this report as Exhibit 99.1.    The C5 Notes bear interest at a rate of 13.8% per annum from the date of issuance (or in the case of the Restated C5 Note, from the date of the First C5 Note), and all such notes are payable at scheduled maturity on June 30, 2023, subject to acceleration in certain circumstances.    The Company's obligations under the Restated Initial Notes and the C5 Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. C5 and each of the holders of the Restated Initial Notes have executed an Amended and Restated Security Agreement in the form attached as an exhibit to the Current Report on Form  8-K  filed on January 4, 2023.    The C5 Notes provide C5 with the right, at any time on or after the date that is five calendar days prior to maturity, to convert any portion of the principal amount of the C5 Notes, together with any accrued and unpaid interest and any other unpaid amounts, into shares of the Company's common stock, par value $0.0001 per share (the " Common Stock "), at a conversion price of $2.00 per share. In the event that any shares of Common Stock are issued upon conversion of the C5 Notes, the Company has agreed to grant specified registration rights to C5.    The foregoing description of the C5 Notes does not purport to be complete and is subject to, and is qualified in its entirety by reference to, the form of C5 Note filed herewith as Exhibit 99.1 and incorporated herein by reference.  " → True
3. "    Between December 14, 2022 and December 16, 2022, IronNet, Inc. (the " Company ") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the " Notes ") to a total of eight lenders. The lenders included GEN Keith B. Alexander (Ret.), the founder, Chief Executive Officer and Chairman of the Board of Directors of the Company. The Notes were also issued to lenders that include VADM John M. McConnell (Ret.), Hon. Michael J. Rogers, Theodore E. Schlein and VADM Jan E. Tighe (Ret.), each of whom is a director of the Company, as well as funds affiliated with Forgepoint Capital (the " Forgepoint Funds "). Donald R. Dixon, a member of the Company's board of directors, is an affiliate of ForgePoint Capital, and the ForgePoint Funds are also beneficial owners of greater than 5% of the Company's common stock.    Each Note has a scheduled maturity date of one year from the date of issuance, except that the Note issued to one lender that is not, and is not affiliated with, a director of the Company has a scheduled maturity date of six months from the date of issuance. The Notes bear interest at a rate of 13.8% per annum, payable at maturity. The Company may prepay the Notes at any time without premium or penalty, and the Notes do not restrict the incurrence of future indebtedness by the Company.    In the event that the Company fails to timely pay amounts due under the Notes or the Company materially defaults in its performance of any other covenant under the Notes or the Security Agreement (as defined below), which default is not cured within 30 days after written notice thereof, then at the option of the lenders, upon the written consent of lenders representing a majority of the outstanding and unpaid principal under the Notes, all unpaid principal, accrued interest and other amounts owing under the Notes shall be immediately due and payable.    In the event that the Company files any petition or action for relief under any bankruptcy, reorganization, insolvency or moratorium law or any other law for the relief of or relating to debtors, or makes any assignment for the benefit of creditors or takes any corporate action in furtherance thereof, or if an involuntary petition is filed against the Company (unless such petition is dismissed or discharged within 60 days) under any bankruptcy statute, or a custodian, receiver, trustee, assignee for the benefit of creditors is appointed to take possession, custody or control of any property of the Company, then all unpaid principal, accrued interest and other amounts owing under the Notes will accelerate and automatically become immediately due and payable.    The Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. The Company entered into a Security Agreement (the " Security Agreements ") with each of the lenders.    The forgoing descriptions of the Notes and the Security Agreements do not purport to be complete and are subject to, and are qualified in their entirety by reference to, the form of Note and form of Security Agreement filed hereto as Exhibits 4.1 and 10.1, respectively, and incorporated herein by reference.  " → True
4. "    Credit Agreement  In connection with the completion of the Merger, on August 4, 2016, Higher One, Inc. ( Higher One ), a Delaware corporation and direct  wholly owned subsidiary of the Company, terminated the existing credit agreement, dated as of October 16, 2012 (as amended, the  Credit Agreement ), among Higher One, as borrower, the Company, Higher One Machines, Inc.,  Higher One Real Estate, Inc. and Higher One Real Estate SP, LLC, as guarantors, Bank of America, N.A. as administrative agent, swingline lender and letter of credit issuer, and the other lenders party thereto, and all commitments thereunder. At the  time of such termination, there were no outstanding revolving loans or outstanding letters of credit under the Credit Agreement. In connection with such termination, all liens on the assets of Higher One, the Company and Higher Ones  subsidiaries securing such facility, together with such subsidiary or Company guarantees, were released and terminated." → True
5. "    On December 28, 2022, the Company and C5 entered into an agreement pursuant to which they have agreed to a mutual exclusivity period through January 31, 2023 to seek to negotiate definitive agreements with respect to a potential offer by C5 to acquire all of the outstanding common stock of the Company not presently owned by C5 and certain of its affiliates (the "Proposed Transaction"). Commencement of the exclusivity period was subject to C5 providing the $2.0 million of bridge financing described in Item 1.01 above. Continuation of the exclusivity period after January 9, 2023 is subject to C5 entering into additional Restated Notes providing $3.5 million of additional bridge financing by January 9, 2023. The January 31, 2023 expiration date is subject to an automatic extension of an additional seven days if C5 provides additional bridge financing sufficient to fund the Company's operations for such seven-day period (up to a maximum additional amount of $3.5 million)." → False
6. "    On June 28, 2018, Fuse Enterprises Inc. (the 'Company') entered into a Share Purchase Agreement (the 'Purchase Agreement') with Fuse Trading Limited, a wholly-owned subsidiary of the Company ('Trading'), and Landbond Home Limited (the 'Purchaser')... pursuant to which the Company agreed to sell to the Purchaser in a private placement (the 'Private Placement') 19,628,050 shares of the Company's common stock... for an aggregate offering price of $6,869,817.60." → False
7. "    The Registrant is filing the exhibits listed in Item 9.01(d) below in connection with the public issuance of Class A-1 Asset Backed Notes, Class A-2 Asset Backed Notes, Class A-3 Asset Backed Notes, Class A-4 Asset Backed Notes and Class B Asset Backed Notes (collectively, the Offered Notes) by CNH Equipment Trust 2011-C, described in the Prospectus dated December 1, 2011 and the Prospectus Supplement dated December 7, 2011." → False

Now classify the following:
"""
# building a numbered list of sentences to append to prompt so model can process a batch of inputs at once
# with clear numbering and formatting for each input example
    numbered_batch = ""
    for i, sentence in enumerate(batch_texts, start=start_idx):
        numbered_batch += f"{i}. \"{sentence}\" →\n"

    final_prompt = few_shot_prompt + numbered_batch

    response = client.chat.completions.create( # maybe use different function rather than chat completion
        model="gpt-3.5-turbo", # wayyy faster than gpt 4; went from 6-7 minutes (only a third complete) to 1 minute (fully complete)
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0
    )

    return response.choices[0].message.content

results = []
batch_size = 7 # Process 5 rows at a time

In [ ]:
len(texts)

NameError: name 'texts' is not defined

In [ ]:
batch_size

7

In [ ]:
print(len(text))
print(len(new))

NameError: name 'text' is not defined

In [ ]:
texts = new['text'].astype(str).tolist()

i = len(texts) - 1
batch = texts[i:i+batch_size]
print(batch)
print()
output = classify_batch(batch, start_idx= i + 1)
print(output)
print(output.strip())
print(output.strip().split('\n'))
#output.strip().split('\n'))

['  Exhibit 99.1 to this Form 8-K is Theravance, Inc.s current  investor presentation and is incorporated by reference herein.  With regard to expense guidance for 2013, we currently anticipate that total 2013 Research and Development expenses plus Selling, General and Administrative expenses will be in the range of $125 million to $135 million.  This guidance does not include stock-based compensation expense or any milestone payments to GlaxoSmithKline plc (GSK) under our long-acting beta 2  agonist (LABA) collaboration with GSK.  Our expectations regarding our expenses for 2013 are forward-looking statements based solely on management estimates utilizing currently available information. As described below under Note Regarding Forward-Looking Statements, investors are cautioned that forward-looking statements are not guarantees of future performance and involve significant risks and uncertainties, and that actual results may differ materially from those projected in the forward-lookin

In [ ]:
# converting the text data into a list, looping through it in batches, classifying each batch,
# printing the output, and collecting all classification results in one list
results=[]
texts = new['text'].astype(str).tolist() # from new df and it is a column

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    output = classify_batch(batch, start_idx= i + 1)
    results.extend(output.strip().split('\n'))
    #print(output)
    #results.extend(output.strip().split('\n'))
#results = [r.strip() for r in results]

In [ ]:
results

In [ ]:
print(len(results))
print(len(new))

In [ ]:
# Convert both predictions and labels to boolean
new['gpt_pred_bool'] = new['gpt_pred'].astype(str).str.lower() == 'true'
new['true_label_bool'] = new['contains_debt_instrument_information'].astype(str).str.lower() == 'true'

from sklearn.metrics import f1_score, classification_report

# Get prediction and true label series
y_pred = new['gpt_pred_bool']
y_true = new['true_label_bool']

# Calculate and print F1 score
f1 = f1_score(y_true, y_pred)
print(f"F1 Score: {f1:.3f}")

# (Optional) See full breakdown: precision, recall, etc.
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

F1 Score: 0.797

Classification Report:
              precision    recall  f1-score   support

       False       0.50      0.04      0.07        55
        True       0.67      0.98      0.80       110

    accuracy                           0.67       165
   macro avg       0.59      0.51      0.43       165
weighted avg       0.61      0.67      0.55       165



In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Generate matrix
cm = confusion_matrix(new['true_label_bool'], new['gpt_pred_bool'])

# Plot it
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["False", "True"])
disp.plot(cmap='Blues')
plt.title("GPT Classification Confusion Matrix")
plt.show()

How does this compare to fine-tuning results?

I'd like to identify false trues and falses, maybe there was human error

*   undersample true values
*   or oversample false values like Tracy did

just go through examples in coral's dataset and compare specific rows there

In [ ]:

pip install transformers accelerate --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "google/flan-t5-base"  # use X-X-large ONLYYYY if you have a good GPU

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

def classify_flan_t5_with_examples(texts, batch_size=7):
    results = []

    few_shot_template = """
The documents here disclose significant, unscheduled events or corporate changes.
Debt financing is when a company borrows money (or is given the right to borrow money) from a collection of lenders. This is primarily done through bonds, notes, loans, credit facilities. Key words include: Term Loan Agreement, Amended and Restated Credit Agreement, Revolving Credit Agreement, Junior Subordinated Notes, Bond, Debt Settlement Agreement.
I want you to think like a financial analyst reviewing filings to determine whether the documents contain material information related to commercial debt. Respond “True” or “False” for each user prompt.

Example 1:
"Alpha Holding Company, LLC Disposition  On January 19, 2017, Concho Resources Inc. (the  Company ), through its wholly owned subsidiary, COG Operating LLC  ( Seller ), entered into a definitive securities purchase agreement (the  Purchase Agreement ) with Plains Pipeline, L.P. ( Purchaser ) to sell 100% of the issued and  outstanding Class A Units of Alpha Holding Company, LLC ( AHC ) (the  Disposition ), for aggregate consideration of approximately $800 million, subject to customary purchase price  adjustments, after closing of the transactions contemplated by the Class B Purchase Agreement (defined below) and repayment of the outstanding borrowings under the credit facility of Alpha Crude Connector, LLC, a wholly owned subsidiary of AHC.  Contemporaneously with the execution of the Purchase Agreement, Purchaser entered into a definitive purchase agreement with Frontier Midstream Solutions, LLC ( Frontier ) to purchase 100% of the issued and outstanding  Class B Units of AHC (the  Class  B Purchase Agreement ). Closing under the Purchase Agreement and the Class B Purchase Agreement is anticipated to occur contemporaneously during the  first half of 2017, subject to customary closing conditions, as further described below. Seller retains the right under the Limited Liability Company Agreement of AHC, subject to certain conditions, to purchase or cause AHC to purchase all of the  Class B Units of AHC prior to closing under the Purchase Agreement and the Class B Purchase Agreement (an  Option Exercise ). In the event of an Option Exercise, Seller will sell all of the Class A Units and  the Class B Units of AHC to Purchaser for an aggregate purchase price of approximately $1.215 billion. The Company expects the net proceeds to Seller in the event of an Option Exercise and sale to Purchaser of all of the Class A Units and  the Class B Units of AHC to be substantially similar to what Seller would otherwise receive under the Purchase Agreement in the absence of an Option Exercise.  Each partys obligation to consummate the Disposition is conditioned upon, among other things, (i) confirmation of the  counterparties representations and warranties as of the closing, (ii) the counterparties performance, in all material respects, of all covenants, (iii) the receipt of all required approvals (including approval under the  Hart-Scott-Rodino Antitrust Improvements Act of 1976), (iv) the absence of legal matters prohibiting the Disposition, and (v) the satisfaction or waiver by Frontier or Purchaser, as applicable, of certain conditions precedent to the  consummation of the transactions contemplated by the Class B Purchase Agreement." -> False

Example 2:
"As previously disclosed, between December 14, 2022 and December 16, 2022, IronNet, Inc. (the "Company") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the "Initial Notes") to a total of eight lenders, and on December 30, 2022, the Company issued a secured promissory note in the principal amount of $2.0 million (the "First C5 Note") to an entity affiliated with C5 Capital Limited ("C5"), a beneficial owner of more than 5% of the Company's outstanding common stock. In connection with the issuance of the First C5 Note, the Company and the holders of the Initial Notes agreed to amend and restate the Initial Notes to be substantially in the form to be issued to C5, which form was attached as an exhibit to the Current Report on Form 8-K filed on January 4, 2023 (the "Restated Initial Notes"). This amendment and restatement occurred on January 11, 2023. The Restated Initial Notes continue to bear interest at a rate of 13.8% per annum from the respective dates of the Initial Notes, and the Restated Initial Notes are payable at scheduled maturity on June 30, 2023. Upon entering into the First C5 Note, the Company and C5 also agreed to amend and restate the First C5 Note. Subsequently, the parties agreed that the amended and restated First C5 Note and any future notes to be issued in connection with additional bridge financing by C5 would be convertible into shares of the Company's common stock. On January 11, 2023, the Company issued to C5 an amended and restated convertible promissory note in the principal amount of $2.0 million (the "Restated C5 Note"). On January 12, 2023, the Company issued to C5 a new secured convertible promissory note in the principal amount of $3.0 million (the "Second C5 Note"). The Restated C5 Note and the Second C5 Note (collectively, the "C5 Notes"), in the aggregate principal amount of $5.0 million, are in substantially the form of secured convertible promissory note attached to this report as Exhibit 99.1. The C5 Notes bear interest at a rate of 13.8% per annum from the date of issuance (or in the case of the Restated C5 Note, from the date of the First C5 Note), and all such notes are payable at scheduled maturity on June 30, 2023, subject to acceleration in certain circumstances. The Company's obligations under the Restated Initial Notes and the C5 Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. C5 and each of the holders of the Restated Initial Notes have executed an Amended and Restated Security Agreement in the form attached as an exhibit to the Current Report on Form 8-K filed on January 4, 2023. The C5 Notes provide C5 with the right, at any time on or after the date that is five calendar days prior to maturity, to convert any portion of the principal amount of the C5 Notes, together with any accrued and unpaid interest and any other unpaid amounts, into shares of the Company's common stock, par value $0.0001 per share (the "Common Stock"), at a conversion price of $2.00 per share. In the event that any shares of Common Stock are issued upon conversion of the C5 Notes, the Company has agreed to grant specified registration rights to C5. The foregoing description of the C5 Notes does not purport to be complete and is subject to, and is qualified in its entirety by reference to, the form of C5 Note filed herewith as Exhibit 99.1 and incorporated herein by reference." -> True

Example 3:
"Between December 14, 2022 and December 16, 2022, IronNet, Inc. (the "Company") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the "Notes") to a total of eight lenders. The lenders included GEN Keith B. Alexander (Ret.), the founder, Chief Executive Officer and Chairman of the Board of Directors of the Company. The Notes were also issued to lenders that include VADM John M. McConnell (Ret.), Hon. Michael J. Rogers, Theodore E. Schlein and VADM Jan E. Tighe (Ret.), each of whom is a director of the Company, as well as funds affiliated with Forgepoint Capital (the "Forgepoint Funds"). Donald R. Dixon, a member of the Company's board of directors, is an affiliate of ForgePoint Capital, and the Forgepoint Funds are also beneficial owners of greater than 5% of the Company's common stock. Each Note has a scheduled maturity date of one year from the date of issuance, except that the Note issued to one lender that is not, and is not affiliated with, a director of the Company has a scheduled maturity date of six months from the date of issuance. The Notes bear interest at a rate of 13.8% per annum, payable at maturity. The Company may prepay the Notes at any time without premium or penalty, and the Notes do not restrict the incurrence of future indebtedness by the Company. In the event that the Company fails to timely pay amounts due under the Notes or the Company materially defaults in its performance of any other covenant under the Notes or the Security Agreement (as defined below), which default is not cured within 30 days after written notice thereof, then at the option of the lenders, upon the written consent of lenders representing a majority of the outstanding and unpaid principal under the Notes, all unpaid principal, accrued interest and other amounts owing under the Notes shall be immediately due and payable. In the event that the Company files any petition or action for relief under any bankruptcy, reorganization, insolvency or moratorium law or any other law for the relief of or relating to debtors, or makes any assignment for the benefit of creditors or takes any corporate action in furtherance thereof, or if an involuntary petition is filed against the Company (unless such petition is dismissed or discharged within 60 days) under any bankruptcy statute, or a custodian, receiver, trustee, assignee for the benefit of creditors is appointed to take possession, custody or control of any property of the Company, then all unpaid principal, accrued interest and other amounts owing under the Notes will accelerate and automatically become immediately due and payable. The Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. The Company entered into a Security Agreement (the "Security Agreements") with each of the lenders. The foregoing descriptions of the Notes and the Security Agreements do not purport to be complete and are subject to, and is qualified in its entirety by reference to, the form of Note and form of Security Agreement filed hereto as Exhibits 4.1 and 10.1, respectively, and incorporated herein by reference." -> True

Example 4:
"Credit Agreement In connection with the completion of the Merger, on August 4, 2016, Higher One, Inc. ( Higher One ), a Delaware corporation and direct wholly owned subsidiary of the Company, terminated the existing credit agreement, dated as of October 16, 2012 (as amended, the Credit Agreement ), among Higher One, as borrower, the Company, Higher One Machines, Inc., Higher One Real Estate, Inc. and Higher One Real Estate SP, LLC, as guarantors, Bank of America, N.A. as administrative agent, swingline lender and letter of credit issuer, and the other lenders party thereto, and all commitments thereunder. At the time of such termination, there were no outstanding revolving loans or outstanding letters of credit under the Credit Agreement. In connection with such termination, all liens on the assets of Higher One, the Company and Higher One’s subsidiaries securing such facility, together with such subsidiary or Company guarantees, were released and terminated." -> True

Example 5:
"On December 28, 2022, the Company and C5 entered into an agreement pursuant to which they have agreed to a mutual exclusivity period through January 31, 2023 to seek to negotiate definitive agreements with respect to a potential offer by C5 to acquire all of the outstanding common stock of the Company not presently owned by C5 and certain of its affiliates (the "Proposed Transaction"). Commencement of the exclusivity period was subject to C5 providing the $2.0 million of bridge financing described in Item 1.01 above. Continuation of the exclusivity period after January 9, 2023 is subject to C5 entering into additional Restated Notes providing $3.5 million of additional bridge financing by January 9, 2023. The January 31, 2023 expiration date is subject to an automatic extension of an additional seven days if C5 provides additional bridge financing sufficient to fund the Company's operations for such seven-day period (up to a maximum additional amount of $3.5 million)." → False

Example 6:
"On June 28, 2018, Fuse Enterprises Inc. (the 'Company') entered into a Share Purchase Agreement (the 'Purchase Agreement') with Fuse Trading Limited, a wholly-owned subsidiary of the Company ('Trading'), and Landbond Home Limited (the 'Purchaser')... pursuant to which the Company agreed to sell to the Purchaser in a private placement (the 'Private Placement') 19,628,050 shares of the Company's common stock... for an aggregate offering price of $6,869,817.60." → False

Example 7:

"The Registrant is filing the exhibits listed in Item 9.01(d) below in connection with the public issuance of Class A-1 Asset Backed Notes, Class A-2 Asset Backed Notes, Class A-3 Asset Backed Notes, Class A-4 Asset Backed Notes and Class B Asset Backed Notes (collectively, the Offered Notes) by CNH Equipment Trust 2011-C, described in the Prospectus dated December 1, 2011 and the Prospectus Supplement dated December 7, 2011." -> False

Example 8:
"On November 14, 2022, IronNet, Inc. (the 'Company') expects to file a registration statement on Form S-3 under the Securities Act of 1933, as amended, to register up to $25,750,000 of the Company's common stock" → Does not have information regarding commericial debt, False


"""


    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        prompts = [few_shot_template.format(text=t.strip()) for t in batch]

        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=5)

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        results.extend([r.strip() for r in decoded])

    return results

In [ ]:
texts = new['text'].astype(str).tolist()
flan_preds = classify_flan_t5_with_examples(texts, batch_size=7)


In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, classification_report

new['flan_pred_raw'] = flan_preds
new['flan_pred'] = new['flan_pred_raw'].str.extract(r'\b(True|False)\b', expand=False)
new['flan_pred_bool'] = new['flan_pred'].str.lower() == 'true'
new['true_label_bool'] = new['contains_debt_instrument_information'].astype(str).str.lower() == 'true'

# Evaluate
valid_rows = new['flan_pred'].notna()
y_pred = new.loc[valid_rows, 'flan_pred_bool']
y_true = new.loc[valid_rows, 'true_label_bool']

print(f"✅ F1 Score (Flan-T5): {f1_score(y_true, y_pred):.3f}")
print("\n📋 Classification Report:\n")
print(classification_report(y_true, y_pred))


NameError: name 'flan_preds' is not defined

In [ ]:
## word cloud lolz

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/CDT Documents/cleaned_banktrack_annotated.csv")

In [ ]:
# Ensure the column is boolean or convert if it's string
df['contains_debt_instrument_information'] = df['contains_debt_instrument_information'].astype(str).str.lower() == 'True'

# Filter for True entries
# debt_texts = df[df['contains_debt_instrument_information']]['text'].dropna().astype(str)

In [ ]:
# Combine into one big string
combined_text = " ".join(debt_texts)


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# Define stopwords — add any financial ones you think are generic
stopwords = set(STOPWORDS)
stopwords.update(["company", "inc", "llc", "corporation", "agreement", "section"])

# Generate word cloud
wordcloud = WordCloud(
    width=1000,
    height=500,
    background_color='white',
    stopwords=stopwords,
    max_words=100,
    colormap='viridis'
).generate(combined_text)

# Plot
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Keywords in Debt-Related 8-K Filings", fontsize=20)
plt.show()


In [ ]:
df['contains_debt_instrument_information'] = df['contains_debt_instrument_information'].astype(str).str.lower() == 'False'
combined_text = " ".join(debt_texts)


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# Define stopwords — add any financial ones you think are generic
stopwords = set(STOPWORDS)
stopwords.update(["company", "inc", "llc", "corporation", "agreement", "section"])

# Generate word cloud
wordcloud = WordCloud(
    width=1000,
    height=500,
    background_color='white',
    stopwords=stopwords,
    max_words=100,
    colormap='viridis'
).generate(combined_text)

# Plot
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Keywords in Debt-Related 8-K Filings", fontsize=20)
plt.show()


## **GPT 4**

In [ ]:
pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 842.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/CDT Documents/cleaned_banktrack_annotated.csv") # undersampled
batch_size = 7
text_batch = df["text"].tolist()[:batch_size]
results = classify_batch(text_batch, start_idx=1)
print(results)


1. True
2. True
3. True
4. True
5. True
6. True
7. True


In [ ]:
batch_size=7
texts = new['text'].astype(str).tolist()

i = len(texts) - 1
batch = texts[i:i+batch_size]
print(batch)
print()
output = classify_batch(batch, start_idx= i + 1)
print(output)
print(output.strip())
print(output.strip().split('\n'))
#output.strip().split('\n'))

In [ ]:
results=[]
texts = new['text'].astype(str).tolist()

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    output = classify_batch(batch, start_idx= i + 1)
    results.extend(output.strip().split('\n'))
    #print(output)
    #results.extend(output.strip().split('\n'))
#results = [r.strip() for r in results]

# **GPT 4 TURBO**

In [ ]:
import openai
def classify_batch(batch_texts, start_idx=1):
    few_shot_prompt = """ I want you to think like a financial analyst

Now classify the following:
"""
# building a numbered list of sentences to append to prompt so model can process a batch of inputs at once
# with clear numbering and formatting for each input example
    numbered_batch = ""
    for i, sentence in enumerate(batch_texts, start=start_idx):
        numbered_batch += f"{i}. \"{sentence}\" →\n"

    final_prompt = few_shot_prompt + numbered_batch

    response = client.chat.completions.create( # maybe use different function rather than chat completion
        model="gpt-4-turbo", # wayyy faster than gpt 4; went from 6-7 minutes (only a third complete) to 1 minute (fully complete)
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0
    )

    return response.choices[0].message.content

results = []
batch_size = 7 # Process 5 rows at a time

In [ ]:
texts = new['text'].astype(str).tolist()

i = len(texts) - 1
batch = texts[i:i+batch_size]
print(batch)
print()
output = classify_batch(batch, start_idx= i + 1)
print(output)
print(output.strip())
print(output.strip().split('\n'))
#output.strip().split('\n'))

['  Exhibit 99.1 to this Form 8-K is Theravance, Inc.s current  investor presentation and is incorporated by reference herein.  With regard to expense guidance for 2013, we currently anticipate that total 2013 Research and Development expenses plus Selling, General and Administrative expenses will be in the range of $125 million to $135 million.  This guidance does not include stock-based compensation expense or any milestone payments to GlaxoSmithKline plc (GSK) under our long-acting beta 2  agonist (LABA) collaboration with GSK.  Our expectations regarding our expenses for 2013 are forward-looking statements based solely on management estimates utilizing currently available information. As described below under Note Regarding Forward-Looking Statements, investors are cautioned that forward-looking statements are not guarantees of future performance and involve significant risks and uncertainties, and that actual results may differ materially from those projected in the forward-lookin

In [ ]:
# converting the text data into a list, looping through it in batches, classifying each batch,
# printing the output, and collecting all classification results in one list
results=[]
texts = new['text'].astype(str).tolist()

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    output = classify_batch(batch, start_idx= i + 1)
    results.extend(output.strip().split('\n'))
    #print(output)
    #results.extend(output.strip().split('\n'))
#results = [r.strip() for r in results]

In [ ]:
# Convert both predictions and labels to boolean
new['gpt_pred_bool'] = new['gpt_pred'].astype(str).str.lower() == 'true'
new['true_label_bool'] = new['contains_debt_instrument_information'].astype(str).str.lower() == 'true'

from sklearn.metrics import f1_score, classification_report

# Get prediction and true label series
y_pred = new['gpt_pred_bool']
y_true = new['true_label_bool']

# Calculate and print F1 score
f1 = f1_score(y_true, y_pred)
print(f"F1 Score: {f1:.3f}")

# (Optional) See full breakdown: precision, recall, etc.
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

F1 Score: 0.818

Classification Report:
              precision    recall  f1-score   support

       False       0.68      0.35      0.46        55
        True       0.74      0.92      0.82       110

    accuracy                           0.73       165
   macro avg       0.71      0.63      0.64       165
weighted avg       0.72      0.73      0.70       165



# **GPT 4.0** **Summary**

In [ ]:
new = pd.read_csv('/content/drive/MyDrive/Group 1: DSSI Summer 2025/Data/summary_banktrak.csv')

In [ ]:
len(new)

44

In [ ]:
from datasets import DatasetDict
def undersample_majority_class(dataset_dict, label_column):
    # Extract train split
    np.random.seed(42)
    train_data = dataset_dict["train"]
    labels = np.array(train_data[label_column])
    # Identify majority and minority classes
    class_counts = np.bincount(labels)
    minority_class = np.argmin(class_counts)
    minority_count = np.min(class_counts)
    # Get indices for both classes
    minority_indices = np.where(labels == minority_class)[0]
    majority_indices = np.where(labels != minority_class)[0] # Q: what does [0] entail in this situation
    # Randomly select subset of majority class to match minority count
    undersampled_majority_indices = np.random.choice(
        majority_indices,
        size=minority_count,  # Match minority count
        replace=False  # No duplicates
    )
    # Combine selected majority + all minority samples
    new_indices = np.concatenate([undersampled_majority_indices, minority_indices])
    # Create new balanced dataset
    balanced_train = train_data.select(new_indices)

    # Create the output DatasetDict, only including 'validation' if it exists
    output_dict = {"train": balanced_train}
    if "validation" in dataset_dict:
        output_dict["validation"] = dataset_dict["validation"]

    return DatasetDict(output_dict)
# Usage
# Convert the pandas DataFrame 'new' to a Hugging Face DatasetDict
from datasets import Dataset
dataset_dict = DatasetDict({"train": Dataset.from_pandas(new)})

dataset = undersample_majority_class(dataset_dict,
    label_column="contains_debt_instrument_information"
)
# Verify
print("Original counts:", np.bincount(dataset_dict["train"]["contains_debt_instrument_information"]))
print("Balanced counts:", np.bincount(dataset["train"]["contains_debt_instrument_information"]))

Original counts: [23 21]
Balanced counts: [21 21]


In [ ]:
# Step 1: Count words in 'text' column
# .astype(str) converts data type of column to string
# .str.split() splits the string by whitespace
# .apply(len) counts number of words in original string
new['word_count'] = new['text'].astype(str).str.split().apply(len)

# Step 2: Filter rows with more than 2000 words
# .copy() copies original dataframe
long_rows = new[new['word_count'] > 2000].copy()

# Step 3: Define the summarization function
# text is string of text I want to summarize
# prompt tells AI model what to do and limits response to 300 words
def summarize_text(text, max_words=300):
    prompt = f"""Summarize the following text in no more than {max_words} words.
Keep only the most essential details and avoid repetition.

Text:
{text}
"""
# sending request to OpenAI API
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    # cleaned summary string is returned
    return response.choices[0].message.content.strip()

# Step 4: Apply summarization to the full text
# each entry in text column in long_rows df is converted to a string
# .apply() runs the summarize_text function on each text entry
long_rows['summary'] = long_rows['text'].astype(str).apply(lambda x: summarize_text(x, max_words=300))

# Assign summaries back to 'new' by matching index
matching_idx = long_rows.index.intersection(new.index)
new.loc[matching_idx, 'summary'] = long_rows.loc[matching_idx, 'summary']
print(long_rows)

Empty DataFrame
Columns: [text, company, cik, item, contains_debt_instrument_information, __index_level_0__, word_count, summary, gpt_pred_bool, true_label_bool]
Index: []


In [ ]:
import openai
def classify_batch(batch_texts, start_idx=1):
    few_shot_prompt = """A Form 8-K is a public document that publicly traded companies must file with the Securities and Exchange Commission (SEC) to disclose significant, unscheduled events or corporate changes. These events are considered material, meaning they could impact the company's financial performance or investor decisions.
        Debt financing is when a company borrows money (or is given the right to borrow money) from a collection of lenders. This is primarily done through bonds, notes, loans, credit facilities. Key words include: Term Loan Agreement, Amended and Restated Credit Agreement, Revolving Credit Agreement, Junior Subordinated Notes, Bond, Debt Settlement Agreement.
        Classification is 'True' if the item contains details about a debt instrument. If the item just references a debt instrument, this column should still be false. There must be some details outside of the name (such as the start date, the amount, the lenders, etc.).
        I want you to think like a financial analyst reviewing SEC Form 8-K filings to determine whether they contain information related to commercial debt. Your response should classify each text with a “True” or “False” label. Do not explain.
Examples:

1. "    Alpha Holding Company, LLC Disposition  On January 19, 2017, Concho Resources Inc. (the  Company ), through its wholly owned subsidiary, COG Operating LLC  ( Seller ), entered into a definitive securities purchase agreement (the  Purchase Agreement ) with Plains Pipeline, L.P. ( Purchaser ) to sell 100% of the issued and  outstanding Class A Units of Alpha Holding Company, LLC ( AHC ) (the  Disposition ), for aggregate consideration of approximately $800 million, subject to customary purchase price  adjustments, after closing of the transactions contemplated by the Class B Purchase Agreement (defined below) and repayment of the outstanding borrowings under the credit facility of Alpha Crude Connector, LLC, a wholly owned subsidiary of AHC.  Contemporaneously with the execution of the Purchase Agreement, Purchaser entered into a definitive purchase agreement with Frontier Midstream Solutions, LLC ( Frontier ) to purchase 100% of the issued and outstanding  Class B Units of AHC (the  Class  B Purchase Agreement ). Closing under the Purchase Agreement and the Class B Purchase Agreement is anticipated to occur contemporaneously during the  first half of 2017, subject to customary closing conditions, as further described below. Seller retains the right under the Limited Liability Company Agreement of AHC, subject to certain conditions, to purchase or cause AHC to purchase all of the  Class B Units of AHC prior to closing under the Purchase Agreement and the Class B Purchase Agreement (an  Option Exercise ). In the event of an Option Exercise, Seller will sell all of the Class A Units and  the Class B Units of AHC to Purchaser for an aggregate purchase price of approximately $1.215 billion. The Company expects the net proceeds to Seller in the event of an Option Exercise and sale to Purchaser of all of the Class A Units and  the Class B Units of AHC to be substantially similar to what Seller would otherwise receive under the Purchase Agreement in the absence of an Option Exercise.  Each partys obligation to consummate the Disposition is conditioned upon, among other things, (i) confirmation of the  counterparties representations and warranties as of the closing, (ii) the counterparties performance, in all material respects, of all covenants, (iii) the receipt of all required approvals (including approval under the  Hart-Scott-Rodino Antitrust Improvements Act of 1976), (iv) the absence of legal matters prohibiting the Disposition, and (v) the satisfaction or waiver by Frontier or Purchaser, as applicable, of certain conditions precedent to the  consummation of the transactions contemplated by the Class B Purchase Agreement.  " → False
2. "    As previously disclosed, between December 14, 2022 and December 16, 2022, IronNet, Inc. (the " Company ") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the " Initial Notes ") to a total of eight lenders, and on December 30, 2022, the Company issued a secured promissory note in the principal amount of $2.0 million (the " First   C5 Note ") to an entity affiliated with C5 Capital Limited (" C5 "), a beneficial owner of more than 5% of the Company's outstanding common stock.    In connection with the issuance of the First C5 Note, the Company and the holders of the Initial Notes agreed to amend and restate the Initial Notes to be substantially in the form to be issued to C5, which form was attached as an exhibit to the Current Report on Form  8-K  filed on January 4, 2023 (the " Restated Initial Notes "). This amendment and restatement occurred on January 11, 2023. The Restated Initial Notes continue to bear interest at a rate of 13.8% per annum from the respective dates of the Initial Notes, and the Restated Initial Notes are payable at scheduled maturity on June 30, 2023.    Upon entering into the First C5 Note, the Company and C5 also agreed to amend and restate the First C5 Note. Subsequently, the parties agreed that the amended and restated First C5 Note and any future notes to be issued in connection with additional bridge financing by C5 would be convertible into shares of the Company's common stock. On January 11, 2023, the Company issued to C5 an amended and restated convertible promissory note in the principal amount of $2.0 million (the " Restated C5 Note "). On January 12, 2023, the Company issued to C5 a new secured convertible promissory note in the principal amount of $3.0 million (the " Second C5 Note "). The Restated C5 Note and the Second C5 Note (collectively, the " C5 Notes "), in the aggregate principal amount of $5.0 million, are in substantially the form of secured convertible promissory note attached to this report as Exhibit 99.1.    The C5 Notes bear interest at a rate of 13.8% per annum from the date of issuance (or in the case of the Restated C5 Note, from the date of the First C5 Note), and all such notes are payable at scheduled maturity on June 30, 2023, subject to acceleration in certain circumstances.    The Company's obligations under the Restated Initial Notes and the C5 Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. C5 and each of the holders of the Restated Initial Notes have executed an Amended and Restated Security Agreement in the form attached as an exhibit to the Current Report on Form  8-K  filed on January 4, 2023.    The C5 Notes provide C5 with the right, at any time on or after the date that is five calendar days prior to maturity, to convert any portion of the principal amount of the C5 Notes, together with any accrued and unpaid interest and any other unpaid amounts, into shares of the Company's common stock, par value $0.0001 per share (the " Common Stock "), at a conversion price of $2.00 per share. In the event that any shares of Common Stock are issued upon conversion of the C5 Notes, the Company has agreed to grant specified registration rights to C5.    The foregoing description of the C5 Notes does not purport to be complete and is subject to, and is qualified in its entirety by reference to, the form of C5 Note filed herewith as Exhibit 99.1 and incorporated herein by reference.  " → True
3. "    Between December 14, 2022 and December 16, 2022, IronNet, Inc. (the " Company ") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the " Notes ") to a total of eight lenders. The lenders included GEN Keith B. Alexander (Ret.), the founder, Chief Executive Officer and Chairman of the Board of Directors of the Company. The Notes were also issued to lenders that include VADM John M. McConnell (Ret.), Hon. Michael J. Rogers, Theodore E. Schlein and VADM Jan E. Tighe (Ret.), each of whom is a director of the Company, as well as funds affiliated with Forgepoint Capital (the " Forgepoint Funds "). Donald R. Dixon, a member of the Company's board of directors, is an affiliate of ForgePoint Capital, and the ForgePoint Funds are also beneficial owners of greater than 5% of the Company's common stock.    Each Note has a scheduled maturity date of one year from the date of issuance, except that the Note issued to one lender that is not, and is not affiliated with, a director of the Company has a scheduled maturity date of six months from the date of issuance. The Notes bear interest at a rate of 13.8% per annum, payable at maturity. The Company may prepay the Notes at any time without premium or penalty, and the Notes do not restrict the incurrence of future indebtedness by the Company.    In the event that the Company fails to timely pay amounts due under the Notes or the Company materially defaults in its performance of any other covenant under the Notes or the Security Agreement (as defined below), which default is not cured within 30 days after written notice thereof, then at the option of the lenders, upon the written consent of lenders representing a majority of the outstanding and unpaid principal under the Notes, all unpaid principal, accrued interest and other amounts owing under the Notes shall be immediately due and payable.    In the event that the Company files any petition or action for relief under any bankruptcy, reorganization, insolvency or moratorium law or any other law for the relief of or relating to debtors, or makes any assignment for the benefit of creditors or takes any corporate action in furtherance thereof, or if an involuntary petition is filed against the Company (unless such petition is dismissed or discharged within 60 days) under any bankruptcy statute, or a custodian, receiver, trustee, assignee for the benefit of creditors is appointed to take possession, custody or control of any property of the Company, then all unpaid principal, accrued interest and other amounts owing under the Notes will accelerate and automatically become immediately due and payable.    The Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. The Company entered into a Security Agreement (the " Security Agreements ") with each of the lenders.    The forgoing descriptions of the Notes and the Security Agreements do not purport to be complete and are subject to, and are qualified in their entirety by reference to, the form of Note and form of Security Agreement filed hereto as Exhibits 4.1 and 10.1, respectively, and incorporated herein by reference.  " → True
4. "    Credit Agreement  In connection with the completion of the Merger, on August 4, 2016, Higher One, Inc. ( Higher One ), a Delaware corporation and direct  wholly owned subsidiary of the Company, terminated the existing credit agreement, dated as of October 16, 2012 (as amended, the  Credit Agreement ), among Higher One, as borrower, the Company, Higher One Machines, Inc.,  Higher One Real Estate, Inc. and Higher One Real Estate SP, LLC, as guarantors, Bank of America, N.A. as administrative agent, swingline lender and letter of credit issuer, and the other lenders party thereto, and all commitments thereunder. At the  time of such termination, there were no outstanding revolving loans or outstanding letters of credit under the Credit Agreement. In connection with such termination, all liens on the assets of Higher One, the Company and Higher Ones  subsidiaries securing such facility, together with such subsidiary or Company guarantees, were released and terminated." → True

Now classify the following:
"""
    # building a numbered list of sentences to append to prompt so model can process a batch of inputs at once
    # with clear numbering and formatting for each input example
    numbered_batch = ""
    for i, sentence in enumerate(batch_texts, start=start_idx):
        numbered_batch += f"{i}. \"{sentence}\" →\n"

    final_prompt = few_shot_prompt + numbered_batch

    response = client.chat.completions.create( # maybe use different function rather than chat completion
        model="gpt-4-turbo", # wayyy faster than gpt 4; went from 6-7 minutes (only a third complete) to 1 minute (fully complete)
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0
    )

    return response.choices[0].message.content

results = []
batch_size = 5 # Process 5 rows at a time

In [ ]:
batch_size

5

In [ ]:
texts = new['text'].astype(str).tolist()

i = len(texts) - 1
batch = texts[i:i+batch_size]
print(batch)
print()
output = classify_batch(batch, start_idx= i + 1)
print(output)
print(output.strip())
print(output.strip().split('\n'))
#output.strip().split('\n'))

["In anticipation of consummating the transactions described in Item 1.01 of this Current Report on Form 8-K, on July 15, 2021, Enstar borrowed $300 million under the Revolving Credit Facility, the proceeds of which will be used to fund in part the purchase under the EGL Agreement and pay costs and expenses associated therewith. Following the draw down, Enstar has approximately $125 million of undrawn borrowing capacity under the Revolving Credit Facility. A summary of the material terms of the Revolving Credit Facility is included in Enstar's Annual Report on  Form 10-K  for the year ended December 31, 2020 filed with the SEC on March 1, 2021 and is incorporated herein by reference"]

True
True
['True']


In [ ]:
# converting the text data into a list, looping through it in batches, classifying each batch,
# printing the output, and collecting all classification results in one list
results=[]
texts = new['text'].astype(str).tolist() # from new df and it is a column

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    output = classify_batch(batch, start_idx= i + 1)
    print(f"--- Raw Output for Batch starting at index {i} ---") # Added print statement
    print(output) # Added print statement
    print("--------------------------------------------------") # Added print statement
    results.extend(output.strip().split('\n'))

# Optionally, you can uncomment the line below to see the collected results after the loop
# results = [r.strip() for r in results]

--- Raw Output for Batch starting at index 0 ---
1. False
2. False
3. True
4. False
5. True
--------------------------------------------------
--- Raw Output for Batch starting at index 5 ---
6. True
7. False
8. False
9. True
10. False
--------------------------------------------------
--- Raw Output for Batch starting at index 10 ---
11. True
12. True
13. False
14. False
15. False
--------------------------------------------------
--- Raw Output for Batch starting at index 15 ---
16. False
17. False
18. True
19. True
20. False
--------------------------------------------------
--- Raw Output for Batch starting at index 20 ---
21. False
22. False
23. True
24. True
25. True
--------------------------------------------------
--- Raw Output for Batch starting at index 25 ---
26. False
27. True
28. False
29. False
30. True
--------------------------------------------------
--- Raw Output for Batch starting at index 30 ---
31. True
32. True
33. False
34. False
35. True
---------------------

In [ ]:
results

['1. False',
 '2. False',
 '3. True',
 '4. False',
 '5. True',
 '6. True',
 '7. False',
 '8. False',
 '9. True',
 '10. False',
 '11. True',
 '12. True',
 '13. False',
 '14. False',
 '15. False',
 '16. False',
 '17. False',
 '18. True',
 '19. True',
 '20. False',
 '21. False',
 '22. False',
 '23. True',
 '24. True',
 '25. True',
 '26. False',
 '27. True',
 '28. False',
 '29. False',
 '30. True',
 '31. True',
 '32. True',
 '33. False',
 '34. False',
 '35. True',
 '36. False',
 '37. True',
 '38. True',
 '39. False',
 '40. True',
 '41. True',
 '42. False',
 '43. True',
 '44. True']

In [ ]:
print(len(set(results)))
print(len(results))

44
44


In [ ]:

# converting the text data into a list, looping through it in batches, classifying each batch,
# printing the output, and collecting all classification results in one list
results=[]
texts = new['text'].astype(str).tolist()

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    output = classify_batch(batch, start_idx= i + 1)
    results.extend(output.strip().split('\n'))
    #print(output)
    #results.extend(output.strip().split('\n'))
#results = [r.strip() for r in results]




In [ ]:

# Calculate and print F1 score
f1 = f1_score(y_true, y_pred)
print(f"F1 Score: {f1:.3f}")

# (Optional) See full breakdown: precision, recall, etc.
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


F1 Score: 0.712

Classification Report:
              precision    recall  f1-score   support

       False       1.00      0.26      0.41        23
        True       0.55      1.00      0.71        21

    accuracy                           0.61        44
   macro avg       0.78      0.63      0.56        44
weighted avg       0.79      0.61      0.56        44

